**NLP**

*CC6205-1 - Otoño 2022*

Autor: Felipe Urrutia Vargas

In [44]:
# !pip install --upgrade pandas

import pandas as pd
import numpy as np
# pd.set_option("max_rows", None)
import pickle
from string import punctuation
import re

from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
# from astropy.visualization import hist
sns.set_theme(style="whitegrid")
plt.rc('axes', titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams.update({'lines.markeredgewidth': 1})
plt.rcParams.update({'errorbar.capsize': 2})
import random

import plotly.express as px

In [21]:
sentiments = "anger fear joy sadness".split()
intensities = "low medium high".split()

root_data = "assignment_1/data"
file_names = {
    "train/anger": "/train/anger-train.txt",
    "train/fear": "/train/fear-train.txt",
    "train/joy": "/train/joy-train.txt",
    "train/sadness": "/train/sadness-train.txt"
}

In [24]:
def txt_to_df(file_name):   
    with open(file_name, encoding="utf-8") as f:
        data = []
        for l in f.read().split("\n")[:-1]:
            o = {}
            for k, v in enumerate(l.split("\t")):
                col_name = "id text sen int".split()[k]
                o[col_name] = v
            data.append(o)
    return pd.DataFrame(data).set_index("id")

In [28]:
df_train = pd.DataFrame()
for sen in sentiments:
    file_name = root_data+file_names[f"train/{sen}"]
    df_sen = txt_to_df(file_name)
    df_train = pd.concat([df_train, df_sen], axis=0)
df_train

,text,sen,int
id,,,
10000,How the fu*k! Who the heck! moved my fridge!.....,anger,high
10001,So my Indian Uber driver just called someone t...,anger,high
10002,@DPD_UK I asked for my parcel to be delivered ...,anger,high
10003,so ef whichever butt wipe pulled the fire alar...,anger,high
10004,Don't join @BTCare they put the phone down on ...,anger,high
...,...,...,...
40855,Common app just randomly logged me out as I wa...,sadness,high
40856,"I'd rather laugh with the rarest genius, in be...",sadness,high
40857,If you #invest in my new #film I will stop ask...,sadness,medium


In [50]:
summ1_data = []
for s in sentiments:
    for i in intensities:
        _sub = df_train[(df_train["sen"] == s) & (df_train["int"] == i)]
        o = {"sen": s, "int": i}
        o["count"] = _sub.shape[0]
        summ1_data.append(o)
summ1_df = pd.DataFrame(summ1_data)

In [53]:
fig = px.bar(summ1_df, x="sen", y="count", color="int", title="Frecuencia de sentimientos e intensidades")
fig.show()
fig.write_html("summ1.html")

In [54]:
df_train[df_train["sen"] == "joy"]

,text,sen,int
id,,,
30000,Just got back from seeing @GaryDelaney in Burs...,joy,high
30001,Oh dear an evening of absolute hilarity I don'...,joy,high
30002,Been waiting all week for this game ❤️❤️❤️ #ch...,joy,high
30003,"@gardiner_love : Thank you so much, Gloria! Yo...",joy,high
30004,I feel so blessed to work with the family that...,joy,high
...,...,...,...
30897,It feels good to get outside for a minute and ...,joy,medium
30898,@r0Ils ppl get triggered over u smiling they'r...,joy,low
30899,@GigaFag @pipertownsend_ snapchat new would be...,joy,medium


In [60]:
#import spacy library
import spacy

!python -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_sm")

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [66]:
vocab_joy = {i: [] for i in intensities}
for i in intensities:
    _sub = df_train[(df_train["sen"] == "joy") & (df_train["int"] == i)]
    for text in _sub["text"]:
        doc = nlp(text)
        for t in doc:
            vocab_joy[i].append(t.text.lower())

In [68]:
import collections

In [73]:
frequencyDict = collections.Counter(vocab_joy["medium"])
sorted(frequencyDict.items(), key=lambda x: x[1], reverse=True)

[('#', 502),
 ('.', 282),
 ('the', 240),
 (',', 200),
 ('i', 190),
 ('to', 164),
 ('a', 158),
 ('and', 122),
 ('you', 113),
 ('of', 110),
 ('!', 102),
 ('is', 100),
 ('it', 88),
 ('this', 82),
 ("'s", 75),
 (' ', 73),
 ('that', 70),
 ('in', 66),
 ('by', 65),
 ("'", 64),
 ('watch', 58),
 ('be', 56),
 ('amazing', 54),
 ('my', 53),
 ('live.ly', 52),
 ('broadcast', 52),
 ('musically', 51),
 ('on', 44),
 ('just', 43),
 ('for', 43),
 ('with', 42),
 ("'m", 41),
 ('me', 41),
 ('at', 40),
 ('so', 39),
 ("n't", 38),
 ('not', 37),
 ('have', 36),
 ('lively', 36),
 ('-', 35),
 (';', 35),
 ('optimism', 34),
 ('but', 34),
 ('good', 33),
 (':', 31),
 ('up', 30),
 ('&', 29),
 ('amp', 29),
 ('your', 29),
 ('happy', 28),
 ('like', 28),
 ('do', 27),
 ('was', 27),
 ('?', 27),
 ('smile', 26),
 ('about', 26),
 ('will', 25),
 ('make', 24),
 ('he', 24),
 ('day', 23),
 ('what', 23),
 ('if', 22),
 ('now', 21),
 ('glee', 21),
 ('we', 21),
 ('all', 21),
 ('can', 21),
 ('out', 21),
 ('*', 21),
 ('hilarious', 20),
 